In [25]:
# Executing this cell does some magic
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import sys
# Add the `src` folder to the system path
src_path = '../src'  # Replace with the actual path to the `src` folder
sys.path.append(src_path)

In [27]:
# Required imports
import pandas as pd
import torch
from torch.utils.data import DataLoader
import os
import numpy as np

# Assuming these scripts are in src/ or current directory
from dataset.rating_dataset import preprocess_data, RatingDataset  # Adjust path as needed
from dataset.anime_data import AnimeData  # Adjust path as needed
from eval.evaluate import evaluate_mse   # Adjust path as needed
from models.genre_baseline import GenreBaselineModel  # Adjust path as needed


## Step 1: Preprocess Data

We'll get `train_dataset`, `val_dataset`, and `test_dataset` using our `preprocess_data` function.


In [28]:
train_dataset, val_dataset, test_dataset = preprocess_data(
    filepath='../data/raw/animelist.csv',
    user_range=10000,  # For example, limit to first 10k users
    include_watching_status=False,
    include_watched_episodes=False,
    zero_strategy='average',
    random_state=42,
    save_interaction_matrix=True
)

len_train = len(train_dataset)
len_val = len(val_dataset)
len_test = len(test_dataset)

print(f"Training samples: {len_train}, Validation samples: {len_val}, Test samples: {len_test}")


Training samples: 2600126, Validation samples: 325016, Test samples: 325016


In [29]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


## Step 2: Initialize AnimeData

We'll load the anime metadata, which will allow us to query genres and compute global genre means.


In [30]:
anime_data = AnimeData(filepath='../data/raw/anime.csv')
print("Anime DataFrame shape:", anime_data.df.shape)
print("Columns:", anime_data.list_columns())


Anime DataFrame shape: (17562, 37)
Columns: ['MAL_ID', 'Name', 'Score', 'Genres', 'English name', 'Japanese name', 'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity', 'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped', 'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1', 'StartDate', 'EndDate']


## Step 3: Initialize and Fit GenreBaselineModel

We'll create an instance of `GenreBaselineModel` and then fit it on the training data. The `fit` method expects a DataFrame with `user_id, anime_id, rating`. We can get that from `train_dataset`.


In [31]:
# Convert train_dataset back to a DataFrame for fitting.
train_df = train_dataset.data  # We stored the original in dataset

model = GenreBaselineModel(anime_data)
model.fit(train_df)


## Step 4: Test Predictions

Let's pick a few random user-item pairs from the validation set and see what the model predicts.


In [32]:
val_df = val_dataset.data
sample = val_df.sample(5, random_state=42)

user_ids = torch.tensor(sample['user_id'].values, dtype=torch.long)
item_ids = torch.tensor(sample['anime_id'].values, dtype=torch.long)
actual_ratings = sample['rating'].values

predictions = model(user_ids, item_ids)

for u, i, actual, pred in zip(user_ids, item_ids, actual_ratings, predictions):
    print(f"User {u.item()}, Item {i.item()} | Actual: {actual}, Predicted: {pred.item()}")


User 9675, Item 8516 | Actual: 7.0, Predicted: 7.014926433563232
User 7364, Item 41930 | Actual: 6.0, Predicted: 7.159160614013672
User 3470, Item 38422 | Actual: 7.3391442155309035, Predicted: 7.390338897705078
User 1504, Item 7311 | Actual: 9.0, Predicted: 7.7423834800720215
User 657, Item 883 | Actual: 5.0, Predicted: 7.25459623336792


We see how the baseline model predicts ratings. It uses the user's mean ratings for the genres associated with that anime, or falls back to the global genre mean.

## Step 5: Evaluate Model

Now we use the `evaluate_mse` function to compute MSE on the validation and test sets.


In [33]:
val_mse = evaluate_mse(model, val_loader, device='cpu')
test_mse = evaluate_mse(model, test_loader, device='cpu')

print("Validation MSE:", val_mse)
print("Test MSE:", test_mse)


Validation MSE: 1.2525918682968895
Test MSE: 1.2607992187064982
